# VQE
近年の量子コンピュータ上で動く、量子コンピュータと古典コンピュータを併用する量子古典ハイブリッドアルゴリズムの代表格です。基本的なアルゴリズムなので、内容を学びます。

## 今回学ぶこと
1. VQEの基礎理論を学ぶ
2. 量子変分回路をBlueqatで実装する

## Blueqatのインストール
pipからBlueqatをインストールします。

In [12]:
!pip install blueqat

---

## 概要
理想的な誤り訂正量子コンピュータを想定した汎用アルゴリズムの他、現在の量子コンピュータのNISQ向けの変分アルゴリズムがあります。

1. 汎用 (グローバー、ショア、位相推定、量子フーリエ変換、HHL、量子サポートベクターマシン、その他)
2. 変分（VQE,QAOA）

VQEは短い量子回路を利用して、既存計算機とのハイブリッドで問題を解くアルゴリズムです。

## 固有値問題
VQEでは与えられた行列（ハミルトニアン）の固有値の期待値を求めます。固有値問題を解くことで様々な問題が解けます。固有値を$E_0$、固有ベクトルを$\mid \psi \rangle$としたとき、

$$
H \mid \psi \rangle = E_0 \mid \psi \rangle
$$

となる$E_0$を求めるのが目的です。

## ハミルトニアンと期待値

問題は、ハミルトニアン$H$と呼ばれるエルミート行列で渡されます。ハミルトニアンはパウリ行列と単位行列からなり、複素数の行列の形をとります。

In [1]:
from blueqat.pauli import X, Y, Z, I

h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
h.to_matrix()

array([[ 3.57+0.j  , -4.56-2.45j],
       [-4.56+2.45j, -1.11+0.j  ]])

ハミルトニアンの期待値は、

$$
\langle \psi \mid H \mid \psi \rangle
$$

のように計算できます。また、ハミルトニアンの期待値はユニタリー行列の線形結合になっているので分解できます。

$$
\langle \psi \mid aH_1 + bH_2 \mid \psi \rangle \\ = \langle \psi \mid aH_1 \mid \psi \rangle + \langle \psi \mid bH_2 \mid \psi \rangle \\ = a\langle \psi \mid H_1 \mid \psi \rangle + b\langle \psi \mid H_2 \mid \psi \rangle
$$

これを使って、例えば、

$$
H = 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1
$$

この式の期待値は下記のように求めることができます。

$$
\langle \psi \mid 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1 \mid \psi \rangle\\
= 1.2\langle \psi \mid X_0 Z_2 \mid \psi \rangle + 2.5 \langle \psi \mid Z_0 X_1 Y_2\mid \psi \rangle - 3.4 \langle \psi \mid Z_2 X_1 \mid \psi \rangle
$$

## ハミルトニアンの期待値とサンプリング
ハミルトニアンの期待値は計算結果のサンプリングから求めることができます。ハミルトニアン$H=Z$のときの期待値は下記のようになります。

$$
\langle \psi \mid Z \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= |\alpha|^2 - |\beta|^2
$$

$|\alpha|^2$と$|\beta|^2$はそれぞれ、0と1のでる確率です。複数回計算を行い、そのサンプル値から期待値を求めます。

通常、ハミルトニアンがXやYの場合にはサンプル値から期待値を求めることができません。その際にはそれぞれ軸の回転を利用してサンプルが取れるように調整をします。

$X$には、$X = HZH$を使います。

$$
\langle \psi \mid X \mid \psi \rangle \\
= \langle \psi \mid HZH \mid \psi \rangle\\
= \langle \psi' \mid Z \mid \psi' \rangle
$$

$Y$には、$Y = RX(-\pi/2) Z RX(\pi/2)$を使います。

$$
\langle \psi \mid Y \mid \psi \rangle \\
= \langle \psi \mid RX(-\pi/2) Z RX(\pi/2) \mid \psi \rangle\\
= \langle \psi'' \mid Z \mid \psi'' \rangle
$$

こちらは、測定の直前に対応する回転ゲートを入れます。

## 量子変分原理
任意の状態ベクトル$\psi(\theta)$において、ハミルトニアンの期待値は下記を満たします。

$$
\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle \geq E_0
$$

VQEではこの量子変分原理を利用し、既存計算機の最適化アルゴリズムを利用し、状態ベクトルを角度パラメータ$\theta$で変えながらできるだけ$E_0$に近づくよう最小値を探していきます。

## Ansatz(アンザッツ)
最小値を効率的に探すための量子回路をAnsatzと呼びます。現在、量子化学にはUCC Ansatzや組合せ最適化問題にはQAOA Ansatzなどの効率的な量子回路が見つかっているため、アプリケーションとして期待されています。Ansatzはある程度分野ごとにルールが決まっていて、ルールに沿って記述します。

## 例題
最後に例題を行ってみます。

1. 回転ゲートの角度をパラメータにしてAnsatzを作ります。（量子）
2. 実行の結果から次を計算します。 $\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle$ （古典）
3. 次の角度パラメータを試します。

今回はAnsatzは任意のものを使います。

```
rx(a)[0].rz(b)[0]
```

というaとbの2つの角度を使って1量子ビットの計算をしてみたいと思います。ハミルトニアンには上で出てきた例を使います。最後にVQEの計算と数値ライブラリのnumpyの計算結果を比較してみます。

In [2]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 2)
        self.step = 1

    def get_circuit(self, params):
        a, b = params
        return Circuit().rx(a)[0].rz(b)[0]

# hamiltonian
h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)

runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy_sparse(result.circuit))

# This is for check
mat = h.to_matrix()
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-4.236663568691615
Result by numpy
-4.450818602983201


これでVQEができました。